In [174]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots


In [175]:
df = pd.read_csv('../data/myfitnesspal/cleaned/mfp_nutrition_cleaned.csv')
print(df)

df = df[df['missing_main_meal'] == False]
daily_cal_df = df.groupby(['Date'])['Calories'].sum().reset_index()
average_daily_cals = daily_cal_df['Calories'].mean()
print(daily_cal_df)

fig = go.Figure()

x_data = daily_cal_df['Date']
y_data = daily_cal_df['Calories']

fig.add_trace(go.Bar(x=x_data,y=y_data))
# fig.add_hline(y=average_daily_cals, line=dict(color='purple', dash='dash'),name ='Average Calories')

fig.add_trace(
    go.Scatter(
        x=[df['Date'].min(), df['Date'].max()],
        y=[average_daily_cals, average_daily_cals],
        mode='lines',
        line=dict(color='purple', dash='dash'),
        name=f'Avg: {average_daily_cals:.0f} kcal'
    )
)
fig.update_layout(title='Daily Calories over time')
fig.show()

            Date       Meal  Calories  Fat (g)  Saturated Fat  \
0     2024-02-28      Lunch     530.7     19.8            3.2   
1     2024-02-29     Dinner      47.4      2.0            1.3   
2     2024-03-01  Breakfast     195.0     10.8            6.9   
3     2024-03-01      Lunch     576.9     34.1            7.5   
4     2024-03-02     Dinner     652.0     30.4           14.2   
...          ...        ...       ...      ...            ...   
1082  2025-06-29      Lunch     449.6     19.9            6.4   
1083  2025-06-29     Snacks     155.0     10.0            8.3   
1084  2025-06-30  Breakfast     229.4     12.1            6.5   
1085  2025-06-30      Lunch     585.4     31.4            5.2   
1086  2025-06-30     Snacks      50.0      3.0            1.5   

      Polyunsaturated Fat  Monounsaturated Fat  Trans Fat  Cholesterol  \
0                     0.8                  3.7        0.0         20.0   
1                     0.0                  0.0        0.0          0.0 

In [176]:
df_daily = pd.read_csv('../data/myfitnesspal/cleaned/mfp_nutrition_daily.csv')
print(df_daily)

df_daily = df_daily[df_daily['missing_main_meal'] == False]
average_daily_cals = df_daily['Calories'].mean()
print(df_daily)
fig = go.Figure()
x_data = df_daily['Date']
y_data = df_daily['Calories']

fig.add_trace(go.Bar(x=x_data,y=y_data))
fig.add_hline(y=average_daily_cals, line=dict(color='purple', dash='dash'),name ='Average Calories')

fig.add_trace(
    go.Scatter(
        x=[df_daily['Date'].min(), df_daily['Date'].max()],
        y=[average_daily_cals, average_daily_cals],
        mode='lines',
        line=dict(color='purple', dash='dash'),
        name=f'Avg: {average_daily_cals:.0f} kcal'
    )
)
fig.show()

           Date  Calories  Protein (g)  Carbohydrates (g)  Fat (g)  \
0    2024-02-28     530.7         31.6               56.1     19.8   
1    2024-02-29      47.4          3.1                4.1      2.0   
2    2024-03-01     771.9         44.2               42.6     44.9   
3    2024-03-02    1055.5         53.9              115.1     43.1   
4    2024-03-07     560.2          4.8               39.2     17.0   
..          ...       ...          ...                ...      ...   
336  2025-06-26    1503.6        100.7              126.0     66.9   
337  2025-06-27    1709.2         99.1              203.1     57.6   
338  2025-06-28     761.4         38.4               82.0     33.5   
339  2025-06-29    1468.4         57.1              142.6     75.0   
340  2025-06-30     864.8         44.2               70.8     46.5   

     Estimated Alcohol (g)  missing_main_meal  Meal  
0                 0.000000               True     1  
1                 0.000000               True     1

# Calculating maintenance calories throughout the time period, based on weight in kg

</br> Mifflin-St Jeor Equation, ref: https://www.calculator.net/calorie-calculator.html
</br> BMR = 10W + 6.25H - 5A -161
</br> W = weight in Kg, H = heigh in cm, A = age


In [177]:
weight_df = pd.read_csv('../data/myfitnesspal/cleaned/mfp_measurements_cleaned.csv')
print(weight_df)

# Creating a column to calculate my maintenance calories at each weight
# Assuming a sedentary lifestyle throughout

weight_df['Date'] = pd.to_datetime(weight_df['Date'],format='%Y-%m-%d')

height_cm = 165
def calculating_bmr(df,height_cm):
    birth_date = pd.to_datetime('1999-05-05')
    df['age'] = (df['Date'] - birth_date).dt.days // 365
    df['bmr'] = (10 * (df['Weight'])) + (6.25 * (height_cm)) - (5 * (df['age'])) - 161
    return df
        
weight_df = calculating_bmr(weight_df,height_cm)

print(weight_df)  


           Date     Weight  Daily_change  weekly_avg_weight  weekly_loss  \
0    2024-02-26  96.600000           NaN                NaN          NaN   
1    2024-02-27  96.600000      0.000000                NaN          NaN   
2    2024-02-28  96.233333     -0.366667                NaN          NaN   
3    2024-02-29  95.866667     -0.366667                NaN          NaN   
4    2024-03-01  95.500000     -0.366667                NaN          NaN   
..          ...        ...           ...                ...          ...   
486  2025-06-26  68.300000      0.500000                NaN          NaN   
487  2025-06-27  67.100000     -1.200000                NaN          NaN   
488  2025-06-28  67.566667      0.466667                NaN          NaN   
489  2025-06-29  68.033333      0.466667          68.014286    -0.565714   
490  2025-06-30  68.500000      0.466667                NaN          NaN   

     Weight lbs  Daily_change lbs  weekly_avg_weight lbs  weekly_loss lbs  
0    212.96

## Exploring calorie deficit vs weight, we can drop all days where there are no calorie inputs


In [178]:
print(df_daily)
print(weight_df)

df_daily['Date'] = pd.to_datetime(df_daily['Date'],format='%Y-%m-%d')
weight_df['Date'] = pd.to_datetime(weight_df['Date'],format='%Y-%m-%d')

joined_df = pd.merge(df_daily, weight_df, on='Date',how='left')
print(joined_df)

joined_df['calorie_deficit'] = joined_df['bmr'] - joined_df['Calories']
print(joined_df)

fig = go.Figure()
x_data = joined_df['Date']
y_data = joined_df['Weight']
y2_data = joined_df['weekly_avg_weight']

fig.add_trace(go.Scatter(x=x_data, y=y_data,name='Weight', line=dict(color='#8CA0D7',width=2)))

fig.add_trace(go.Scatter(x=x_data, y=y2_data, mode='markers',name='Weekly average weight', line=dict(color='#91C4F2')))

fig.update_layout(
    yaxis=dict(title='Weight (kg)', range=[60,100]), 
    template='plotly_white', 
    font=dict(size=14), title=dict(text='Weight Over Time', x=0.5, font=dict(size=16))
    )

fig.show()

# Calculating the correlation coefficient between calorie deficit and weight using Pearson correlation coefficient
# Creating cumulative sum of data to support correlation

joined_df['cumulative_deficit'] = (joined_df['calorie_deficit'] * -1).cumsum()

correlation = joined_df['Weight'].corr(joined_df['cumulative_deficit'])
print(correlation)
print(joined_df)

fig = go.Figure()

x_data = joined_df['Date']
y_data = joined_df['Weight']
y2_data = joined_df['cumulative_deficit']

fig.add_trace(go.Scatter(x=x_data, y=y_data, name='Weight (kg)', line=dict(color='#8CA0D7',width=2)))
fig.add_trace(go.Scatter(x=x_data, y=y2_data, name='Cumulative Calorie Deficit (kcal)',yaxis='y2', line=dict(color='#9D79BC',width=2,dash='dash')))

fig.add_annotation(xref='paper', yref='paper', x=0.99, y=0.95, text=f"Pearson r = {correlation:.2f}, strong correlation",showarrow=False,
                   font=dict(size=14,color='black'), bgcolor='white',bordercolor='black',borderwidth=1)

fig.update_layout(
    yaxis=dict(title='Weight (kg)', range=[60,100]), 
    yaxis2=dict(title='Calorie deficit (kcal)',overlaying='y', side='right', range=[-40000,0]),
    template='plotly_white', 
    font=dict(size=14), title=dict(text='Weight and Calorie Deficit Over Time', x=0.5, font=dict(size=16))
    )
fig.show()






           Date  Calories  Protein (g)  Carbohydrates (g)  Fat (g)  \
6    2024-03-19    1203.9         71.9              128.2     49.2   
15   2024-06-02    1430.1        112.4              104.3     67.8   
16   2024-06-03    1497.6        117.4              170.4     43.5   
17   2024-06-04    1146.9        104.9               70.4     50.9   
18   2024-06-05    1529.6        119.9              109.9     70.8   
..          ...       ...          ...                ...      ...   
335  2025-06-25    1542.5        104.0              136.1     66.4   
336  2025-06-26    1503.6        100.7              126.0     66.9   
337  2025-06-27    1709.2         99.1              203.1     57.6   
338  2025-06-28     761.4         38.4               82.0     33.5   
339  2025-06-29    1468.4         57.1              142.6     75.0   

     Estimated Alcohol (g)  missing_main_meal  Meal  
6                      0.0              False     2  
15                     0.0              False     4

0.9501684550155067
          Date  Calories  Protein (g)  Carbohydrates (g)  Fat (g)  \
0   2024-03-19    1203.9         71.9              128.2     49.2   
1   2024-06-02    1430.1        112.4              104.3     67.8   
2   2024-06-03    1497.6        117.4              170.4     43.5   
3   2024-06-04    1146.9        104.9               70.4     50.9   
4   2024-06-05    1529.6        119.9              109.9     70.8   
..         ...       ...          ...                ...      ...   
275 2025-06-25    1542.5        104.0              136.1     66.4   
276 2025-06-26    1503.6        100.7              126.0     66.9   
277 2025-06-27    1709.2         99.1              203.1     57.6   
278 2025-06-28     761.4         38.4               82.0     33.5   
279 2025-06-29    1468.4         57.1              142.6     75.0   

     Estimated Alcohol (g)  missing_main_meal  Meal     Weight  Daily_change  \
0                      0.0              False     2  93.600000     -0.01

In [179]:

df_daily['Date'] = pd.to_datetime(df_daily['Date'],format='%Y-%m-%d')
weight_df['Date'] = pd.to_datetime(weight_df['Date'],format='%Y-%m-%d')

joined_df = pd.merge(df_daily, weight_df, on='Date',how='left')
# joined_df['calorie_deficit'] = joined_df['bmr'] - joined_df['Calories']
# joined_df['cumulative_deficit'] = (joined_df['calorie_deficit'] * -1).cumsum()
joined_df['Calories weekly rolling'] = joined_df['Calories'].rolling(window=7).mean()


fig = go.Figure()

joined_df['Calorie goal'] = joined_df['bmr'] - 300

x_data = joined_df['Date']
y_data = joined_df['Calories']
y2_data = joined_df['Calorie goal']
y3_data = joined_df['Calories weekly rolling']

fig.add_trace(go.Bar(x=x_data, y=y_data, name='Calories (kcal)', marker=dict(color='#8CA0D7')))
fig.add_trace(go.Scatter(x=x_data, y=y2_data, name='Goal Calories (kcsl)', line=dict(color='#9D79BC',width=2,dash='dash')))
fig.add_trace(go.Scatter(x=x_data, y=y3_data, name='rolling avg', line=dict(color='blue',width=2,dash='dash')))
fig.update_layout(plot_bgcolor='white',paper_bgcolor='white')
fig.show()

# Macronutrient Breakdown


In [180]:
# print(df_daily)
print(df)

df_daily = pd.read_csv('../data/myfitnesspal/cleaned/mfp_nutrition_daily.csv')
# only interested in days with consistent trackin
df_daily = df_daily[df_daily['missing_main_meal']==False]

avg_carbs = df_daily['Carbohydrates (g)'].mean()
avg_protein = df_daily['Protein (g)'].mean()
avg_fat = df_daily['Fat (g)'].mean()

fig = go.Figure()

fig.add_trace(go.Pie(labels=['Carbs (g)','Protein (g)','Fats (g)'],
                     values = [avg_carbs,avg_protein, avg_fat],
                     hoverinfo='label+percent',
                     textinfo='label+value',
                     marker=dict(colors=['91C4F2','9D79BC','8CA0D7'])))

fig.update_layout(title_text='Average Macronutrient Breakdown')
fig.show()


#  Creating subplots for broken down by meal

fig = go.Figure()
meals = df['Meal'].unique()

fig = make_subplots(rows=1, cols=4, specs=[[{'type':'domain'},{'type':'domain'},{'type':'domain'},{'type':'domain'}]], subplot_titles=meals)
annotations = []
# annotation_spacing
x = 0.1

for i, meal in enumerate(meals, start=1):
    df_meal = df[df['Meal'] == meal]
    avg_carbs = df_meal['Carbohydrates (g)'].mean()
    avg_protein = df_meal['Protein (g)'].mean()    
    avg_fat = df_meal['Fat (g)'].mean()
    avg_calories = df_meal['Calories'].mean()
    
    print(avg_carbs,avg_protein,avg_fat)
    
    fig.add_trace(go.Pie(labels=['Carbs (g)','Protein (g)','Fats (g)'],
                         values=[avg_carbs,avg_protein,avg_fat],
                         name=meal,
                         hoverinfo='label+percent',
                         textinfo='label+value',
                         marker=dict(colors=['#91C4F2','#9D79BC','#8CA0D7'])
                         ), row=1, col=i)
    # spacing for annotations
    
    annotations.append(dict(
        x=x,  # x-position normalized between 0 and 1
        y=-0.15,                       # slightly below the plot area
        text=f'Avg Calories: {avg_calories:.0f}kcal',
        showarrow=False,
        xref='paper',
        yref='paper',
        font=dict(size=12),
        align='center'
    ))
    x+=0.27

fig.update_layout(title_text='Average Macronutrient Breakdown by Meal', annotations=annotations, margin=dict(t=100,b=100))
fig.show()







            Date       Meal  Calories  Fat (g)  Saturated Fat  \
10    2024-03-19  Breakfast     651.7     28.6           10.6   
11    2024-03-19     Dinner     552.2     20.6            8.8   
24    2024-06-02  Breakfast      78.0      4.3            2.8   
25    2024-06-02     Dinner     812.8     40.5           12.9   
26    2024-06-02      Lunch     505.7     22.0            3.8   
...          ...        ...       ...      ...            ...   
1079  2025-06-28      Lunch     483.0     19.8            9.1   
1080  2025-06-29  Breakfast     278.4     13.7            6.0   
1081  2025-06-29     Dinner     585.4     31.4            5.2   
1082  2025-06-29      Lunch     449.6     19.9            6.4   
1083  2025-06-29     Snacks     155.0     10.0            8.3   

      Polyunsaturated Fat  Monounsaturated Fat  Trans Fat  Cholesterol  \
10                    2.8                  5.1        0.0        137.7   
11                    0.4                  1.6        0.0         19.5 

14.471014492753621 9.412318840579712 9.156884057971014
55.583397683397685 36.0019305019305 27.54594594594595
47.60932835820896 32.62723880597016 22.15186567164179
27.32857142857143 9.773714285714286 13.005714285714285


# Weekly breakdowns


In [181]:
# Breaking these down by week
df_daily = pd.read_csv('../data/myfitnesspal/cleaned/mfp_nutrition_daily.csv')
print(df_daily)
df_daily['Date'] = pd.to_datetime(df['Date'],format='%Y-%m-%d')
df_daily = df_daily.set_index('Date')
weekly_df = df_daily.resample('W').mean().reset_index()[['Date','Calories','Protein (g)','Fat (g)','Carbohydrates (g)']]

print(weekly_df)

# fig = go.Figure()

# x_data = weekly_df['Date']
# y1_data = weekly_df['Carbohydrates (g)']
# y2_data = weekly_df['Protein (g)']
# y3_data = weekly_df['Fat (g)']


# # CPF = ['#91C4F2','#9D79BC','#8CA0D7']

# fig.add_trace(go.Bar(x=x_data,y=y1_data,name='Carbohydrates (g)', marker=dict(color='#91C4F2') ))
# fig.add_trace(go.Bar(x=x_data,y=y2_data,name='Protein (g)', marker=dict(color='#9D79BC') ))
# fig.add_trace(go.Bar(x=x_data,y=y3_data,name='Fat (g)', marker=dict(color='#8CA0D7') ))

# fig.update_layout(barmode='stack', title='Weekly Macronutrient Intake', xaxis_title='Date',yaxis_title='Grams',legend_title='Macronutrient')


           Date  Calories  Protein (g)  Carbohydrates (g)  Fat (g)  \
0    2024-02-28     530.7         31.6               56.1     19.8   
1    2024-02-29      47.4          3.1                4.1      2.0   
2    2024-03-01     771.9         44.2               42.6     44.9   
3    2024-03-02    1055.5         53.9              115.1     43.1   
4    2024-03-07     560.2          4.8               39.2     17.0   
..          ...       ...          ...                ...      ...   
336  2025-06-26    1503.6        100.7              126.0     66.9   
337  2025-06-27    1709.2         99.1              203.1     57.6   
338  2025-06-28     761.4         38.4               82.0     33.5   
339  2025-06-29    1468.4         57.1              142.6     75.0   
340  2025-06-30     864.8         44.2               70.8     46.5   

     Estimated Alcohol (g)  missing_main_meal  Meal  
0                 0.000000               True     1  
1                 0.000000               True     1